In [1]:
import pandas as pd
import tensorflow as tf

df = pd.read_csv('./data/creditcard.csv')

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
fraud = df[df.Class == 1]
normal = df[df.Class == 0]

In [4]:
X_train = pd.concat([fraud.sample(frac=0.8, 
                                  random_state=0), 
                     normal.sample(frac=0.8,
                                  random_state=0)], 
                    axis=0)
X_test = df.loc[~df.index.isin(X_train.index)]

In [5]:
from sklearn.utils import shuffle

X_train = shuffle(X_train, random_state=0)
X_test = shuffle(X_test, random_state=0)

In [6]:
import numpy as np 

y_train = np.zeros((X_train.shape[0], 2),  dtype=float)
y_train[:, 1] = X_train.Class
y_train[:, 0] = 1 - y_train[:, 1]

y_test = np.zeros((X_test.shape[0], 2), dtype=float)
y_test[:, 1] = X_test.Class
y_test[:, 0] = 1 - y_test[:, 1]

In [7]:
ratio = len(X_train) / len(X_train[X_train.Class == 1]) 

#y_train[:, 1] *= ratio

In [8]:
features = X_train.columns.values
for feature in features:
    mean = df[feature].mean()
    std = df[feature].std()
    X_train.loc[:, feature] = \
        (X_train[feature] - mean) / std
    X_test.loc[:, feature] = \
        (X_test[feature] - mean) / std

In [9]:
fields = ['Class']
X_train = X_train.drop(fields, axis=1)
X_test = X_test.drop(fields, axis=1)

In [10]:
split = (X_train.shape[0] * 7) // 8

inputX = X_train.as_matrix()[:split]
inputX_valid = X_train.as_matrix()[split:]
inputX_test = X_test.as_matrix()

inputY = y_train[:split]
inputY_valid = y_train[split:]
inputY_test = y_test

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [11]:
hidden_nodes1 = 60
hidden_nodes2 = 30
hidden_nodes3 = 15

In [12]:
input_nodes = inputX.shape[1]

x = tf.placeholder(tf.float32, [None, input_nodes])
y_ = tf.placeholder(tf.float32, [None, 2])
pkeep = tf.placeholder(tf.float32)

In [13]:
W1 = tf.Variable(tf.zeros([input_nodes, 
                           hidden_nodes1]))
b1 = tf.Variable(tf.zeros([hidden_nodes1]))
y1 = tf.nn.sigmoid(tf.matmul(x, W1) + b1)

In [14]:
W2 = tf.Variable(tf.zeros([hidden_nodes1, 
                           hidden_nodes2]))
b2 = tf.Variable(tf.zeros([hidden_nodes2]))
y2 = tf.nn.sigmoid(tf.matmul(y1, W2) + b2)

In [15]:
W3 = tf.Variable(tf.zeros([hidden_nodes2, 
                           hidden_nodes3])) 
b3 = tf.Variable(tf.zeros([hidden_nodes3]))
y3 = tf.nn.sigmoid(tf.matmul(y2, W3) + b3)
y3 = tf.nn.dropout(y3, pkeep)

In [16]:
W4 = tf.Variable(tf.zeros([hidden_nodes3, 2])) 
b4 = tf.Variable(tf.zeros([2]))
y4 = tf.nn.softmax(tf.matmul(y3, W4) + b4)
y = y4

In [17]:
learning_rate = 0.005
cost = -tf.reduce_sum(y_ * tf.log(y))
optimizer = tf.train.AdamOptimizer(0.005)
optimizer = optimizer.minimize(cost)

In [18]:
labels = tf.argmax(y_, 1)
predictions = tf.argmax(y, 1)
correct = tf.equal(labels, predictions)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [19]:
_, pre_op = tf.metrics.precision(labels, predictions)
_, rec_op = tf.metrics.recall(labels, predictions)

In [20]:
def print_epoch(epoch):
    trn_res = sess.run([accuracy, pre_op, rec_op, cost],
                       feed_dict={x: inputX, 
                                  y_: inputY,
                                  pkeep: dropout})
    vld_res = sess.run([accuracy, pre_op, rec_op, cost], 
                       feed_dict={x: inputX_valid, 
                                  y_: inputY_valid,
                                  pkeep: 1})
    test_res = sess.run([accuracy, pre_op, rec_op, cost], 
                       feed_dict={x: inputX_test, 
                                  y_: inputY_test,
                                  pkeep: 1})    
    print('Epoch: {}'.format(epoch))
    print('Training (acc={:.4f}, prec={:.4f}, '
          'recall={:.4f}, cost={:.1f})'.format(*trn_res))
    print('Validation (acc={:.4f}, prec={:.4f}, '
          'recall={:.4f}, cost={:.1f})'.format(*vld_res))
    print('Test (acc={:.4f}, prec={:.4f}, '
          'recall={:.4f}, cost={:.1f})\n'.format(*test_res))

In [21]:
n_epochs = 100
n_samples = inputY.shape[0]
batch_size = 2048
n_batches = n_samples // batch_size
dropout = 0.9

In [22]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    for epoch in range(n_epochs): 
        for batch in range(n_batches):
            idx1 = batch * batch_size
            idx2 = (1 + batch) * batch_size
            batch_x = inputX[idx1:idx2]
            batch_y = inputY[idx1:idx2]

            sess.run([optimizer], feed_dict={
                x: batch_x, y_: batch_y, pkeep: dropout})
            
        if epoch % 5 == 0:
            print_epoch(epoch)

Epoch: 0
Training (acc=0.9983, prec=0.0000, recall=0.0000, cost=2861.1)
Validation (acc=0.9982, prec=0.0000, recall=0.0000, cost=404.0)
Test (acc=0.9983, prec=0.0000, recall=0.0000, cost=787.5)

Epoch: 5
Training (acc=0.9983, prec=0.0000, recall=0.0000, cost=2100.6)
Validation (acc=0.9982, prec=0.0000, recall=0.0000, cost=301.8)
Test (acc=0.9983, prec=0.0000, recall=0.0000, cost=581.7)

Epoch: 10
Training (acc=0.9983, prec=0.0000, recall=0.0000, cost=1492.8)
Validation (acc=0.9982, prec=0.0000, recall=0.0000, cost=216.6)
Test (acc=0.9983, prec=0.0000, recall=0.0000, cost=413.1)

Epoch: 15
Training (acc=0.9983, prec=0.0000, recall=0.0000, cost=1047.9)
Validation (acc=0.9982, prec=0.0000, recall=0.0000, cost=153.2)
Test (acc=0.9983, prec=0.0000, recall=0.0000, cost=287.7)

Epoch: 20
Training (acc=0.9983, prec=0.0000, recall=0.0000, cost=838.8)
Validation (acc=0.9982, prec=0.0000, recall=0.0000, cost=126.0)
Test (acc=0.9983, prec=0.0000, recall=0.0000, cost=230.4)

Epoch: 25
Training (acc

In [23]:
X_train = pd.concat([normal.sample(frac=0.8,
                                   random_state=0)], 
                    axis=0)
X_test = df.loc[~df.index.isin(X_train.index)]